In [1]:
import time

import pandas as pd
import requests

# 1. 초기 설정
base_url = "http://apis.data.go.kr/1613000/RTMSDataSvcOffiRent/getRTMSDataSvcOffiRent"
service_key = requests.utils.unquote(
    "8afa4c269ca52320f0ba72cd7d98b8d8003a8c88cafcb41d001dddbca8debf89"
)

lawd_codes = {
    "서울(종로)": "11110",
    "충남(아산)": "44210",
    "전남(무안)": "46830",
    "전북(완주)": "45710",
    "충남(홍성)": "44800",
    "경북(안동)": "47170",
}

months = [f"2025{m:02d}" for m in range(6, 12)]  # 2025-06 ~ 2025-11
all_data = []

session = requests.Session()

for deal_ymd in months:
    print(f"{deal_ymd} 데이터 수집 시작...")

    for region_name, code in lawd_codes.items():
        page = 1
        total_added = 0

        while True:
            params = {
                "serviceKey": service_key,  # ✅ unquote 제거 권장
                "LAWD_CD": code,
                "DEAL_YMD": deal_ymd,
                "pageNo": page,
                "numOfRows": 1000,
                "_type": "json",
            }

            try:
                response = session.get(base_url, params=params, timeout=20)

                if response.status_code != 200:
                    print(
                        f"  서버 응답 에러: {response.status_code} ({region_name} {code})"
                    )
                    break

                data_dict = response.json()
                header = data_dict.get("response", {}).get("header", {})
                result_code = str(header.get("resultCode", "")).strip()
                result_msg = str(header.get("resultMsg", "")).strip()
                if result_code not in ("00", "000"):
                    print(
                        f"  ❌ API 에러 ({region_name} {code}, {deal_ymd}, page={page}): "
                        f"{result_code} / {result_msg}"
                    )
                    break

                body = data_dict.get("response", {}).get("body", {}) or {}
                items = body.get("items", {}) or {}
                item_list = items.get("item")

                # ✅ 더 이상 데이터 없으면 종료
                if not item_list:
                    if page == 1:
                        print(f"  [{deal_ymd}] {region_name}({code}): 데이터 없음")
                    break

                # ✅ 단건(dict) → list로 통일
                if isinstance(item_list, dict):
                    item_list = [item_list]

                # ✅ row마다 메타정보 추가(나중에 분석 편함)
                for it in item_list:
                    it["_deal_ymd"] = deal_ymd
                    it["_region_name"] = region_name
                    it["_lawd_cd"] = code
                    it["_pageNo"] = page

                all_data.extend(item_list)
                total_added += len(item_list)
                print(
                    f"페이지 {page}  [{deal_ymd}] 지역 {region_name}: {len(item_list)}건 추가"
                )

                # 다음 페이지로
                page += 1

                # API 서버 부하 방지
                time.sleep(0.2)

            except Exception as e:
                print(f"  [{deal_ymd}] {region_name}({code}) page={page} 오류: {e}")
                break

        if total_added > 0:
            print(
                f"  ✅ [{deal_ymd}] {region_name}({code}): 총 {total_added}건 수집 완료"
            )

print("-" * 30)
print(f"수집 종료: 총 {len(all_data)}건의 데이터가 리스트에 저장되었습니다.")


202506 데이터 수집 시작...
페이지 1  [202506] 지역 서울(종로): 154건 추가
  ✅ [202506] 서울(종로)(11110): 총 154건 수집 완료
페이지 1  [202506] 지역 충남(아산): 39건 추가
  ✅ [202506] 충남(아산)(44210): 총 39건 수집 완료
  [202506] 전남(무안)(46830): 데이터 없음
  [202506] 전북(완주)(45710): 데이터 없음
페이지 1  [202506] 지역 충남(홍성): 7건 추가
  ✅ [202506] 충남(홍성)(44800): 총 7건 수집 완료
페이지 1  [202506] 지역 경북(안동): 52건 추가
  ✅ [202506] 경북(안동)(47170): 총 52건 수집 완료
202507 데이터 수집 시작...
페이지 1  [202507] 지역 서울(종로): 172건 추가
  ✅ [202507] 서울(종로)(11110): 총 172건 수집 완료
페이지 1  [202507] 지역 충남(아산): 49건 추가
  ✅ [202507] 충남(아산)(44210): 총 49건 수집 완료
  [202507] 전남(무안)(46830): 데이터 없음
  [202507] 전북(완주)(45710): 데이터 없음
페이지 1  [202507] 지역 충남(홍성): 12건 추가
  ✅ [202507] 충남(홍성)(44800): 총 12건 수집 완료
페이지 1  [202507] 지역 경북(안동): 42건 추가
  ✅ [202507] 경북(안동)(47170): 총 42건 수집 완료
202508 데이터 수집 시작...
페이지 1  [202508] 지역 서울(종로): 155건 추가
  ✅ [202508] 서울(종로)(11110): 총 155건 수집 완료
페이지 1  [202508] 지역 충남(아산): 46건 추가
  ✅ [202508] 충남(아산)(44210): 총 46건 수집 완료
  [202508] 전남(무안)(46830): 데이터 없음
  [202508] 전북(완주)(45710): 데이터 없

In [2]:
df = pd.DataFrame(all_data)
print("df shape:", df.shape)
print(df.head())


df shape: (1547, 22)
  buildYear contractTerm contractType  dealDay  dealMonth  dealYear deposit  \
0      2021                                 17          6      2025   1,000   
1      2019  25.06~26.05           신규       30          6      2025   1,000   
2      2004  25.07~27.07           신규       17          6      2025   2,000   
3      2008  25.08~26.08           갱신        4          6      2025  33,000   
4      2022  25.08~26.08           신규       19          6      2025   1,000   

   excluUseAr  floor  jibun  ...  preDeposit preMonthlyRent  sggCd sggNm  \
0       33.71      5  126-2  ...                             11110   종로구   
1       19.83      6  193-1  ...                             11110   종로구   
2       56.48      4     73  ...                             11110   종로구   
3       39.54      2      9  ...      33,000                 11110   종로구   
4       21.99      9   66-6  ...                             11110   종로구   

   umdNm useRRRight _deal_ymd _region_name _law

In [ ]:
import pandas as pd

# 1. 안전한 컬럼명 통합 및 변경
# 오피스텔(offiNm)과 아파트(aptNm) 컬럼이 섞여 있을 경우를 대비합니다.
if "offiNm" in df.columns and "aptNm" in df.columns:
    df["단지명"] = df["offiNm"].fillna(df["aptNm"])
elif "offiNm" in df.columns:
    df["단지명"] = df["offiNm"]
elif "aptNm" in df.columns:
    df["단지명"] = df["aptNm"]

# 나머지 주요 컬럼 이름 변경 (존재할 때만 실행)
rename_dict = {
    "sggCd": "행정코드",
    "deposit": "보증금",
    "monthlyRent": "월세",
    "dealYear": "계약년",
    "dealMonth": "계약월",
    "buildYear": "건축년도",
}

# 실제 df에 존재하는 컬럼만 골라서 변경
actual_rename = {k: v for k, v in rename_dict.items() if k in df.columns}
df.rename(columns=actual_rename, inplace=True)

# 2. 숫자 데이터 정제 (쉼표 제거 및 강제 형변환)
# '보증금'이나 '월세' 컬럼이 이미 한글로 바뀌었는지 확인 후 처리
target_cols = [c for c in ["보증금", "월세"] if c in df.columns]
for col in target_cols:
    df[col] = pd.to_numeric(
        df[col].astype(str).str.replace(",", ""), errors="coerce"
    ).fillna(0)

# 3. 실질 주거비 계산 (LaTeX 공식 적용)
# $$RealCost = MonthlyRent + \frac{Deposit \times 0.055}{12}$$
if "보증금" in df.columns and "월세" in df.columns:
    df["실질주거비"] = df["월세"] + (df["보증금"] * 0.055 / 12)
    df["실질주거비"] = df["실질주거비"].round(0).astype(int)

# 4. 지역명 매핑 (행정코드 앞 5자리 기준)
region_map = {
    "11110": "서울(종로)",
    "44210": "충남(아산)",
    "46830": "전남(무안)",
    "45710": "전북(완주)",
    "44800": "충남(홍성)",
    "47170": "경북(안동)",
}
df["지역명"] = df["행정코드"].astype(str).str[:5].map(region_map)

# 5. 결과 확인 (NaN이 아닌 지역들 위주로 샘플링)
print("✅ 정제 완료. 데이터 샘플:")
display(
    df[df["지역명"].notna()][["지역명", "단지명", "보증금", "월세", "실질주거비"]].head(
        10
    )
)

✅ 정제 완료. 데이터 샘플:


,지역명,단지명,보증금,월세,실질주거비
0,서울(종로),이지마루종로,1000,150,155
1,서울(종로),BS타워,1000,75,80
2,서울(종로),경희궁의 아침 4단지,2000,160,169
3,서울(종로),광화문 스페이스본,33000,0,151
4,서울(종로),휴스턴오피스텔 창경궁 C,1000,110,115
5,서울(종로),경희궁자이,1000,103,108
6,서울(종로),아이비씨 호텔 앤 오피스텔,1000,66,71
7,서울(종로),경희궁자이,27000,0,124
8,서울(종로),한라 운종가,3000,82,96
9,서울(종로),한라 운종가,1000,105,110


In [ ]:
# 가상의 순이동률 데이터프레임(df_migration)이 있다고 가정할 때
# 두 데이터를 '행정코드'와 '계약월'을 기준으로 결합
df_final = pd.merge(df_refined, df_migration, on=["행정코드", "계약월"], how="inner")